In [1]:
import requests 
from bs4 import BeautifulSoup
from pprint import pprint
import pandas as pd

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Base URL
base_url = "http://www.racquetfinder.com/?name=&manufacturer={}&hsMin=&hsMax=&lMin=&lMax=&wMin=&wMax=&swMin=&swMax=&fMin=&fMax=&bpMin=&bpMax=&bwMin=&bwMax=&mains=&crosses=&currentcheckbox=ASICS&current=Y"

# List of racquet manufacturers
manufacturer_list = ['Babolat', 'Yonex', 'Wilson', 'Head', 'Dunlop', 'Prince', 'Tecnifibre', 'Volkl', 'ProKennex']

# Creating URLs
urls = [base_url.format(manufacturer) for manufacturer in manufacturer_list]

# List to store individual DataFrames
dfs = []

# Iterating over each URL
for url in urls:
    # Making HTTP request
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extracting desired data
    racquet_list = soup.find_all("div", class_="product cf")
    
    # Iterating over each racquet in the list
    for racquet in racquet_list:
        name = racquet.find('div', class_='rac_name').get_text(strip=True)
        head_size = racquet.find("th", string="Head Size: ").find_next('td').get_text(strip=True)
        balance = racquet.find('th', string='Balance: ').find_next('td').get_text(strip=True)
        weight = racquet.find('th', string='Swing Weight: ').find_next('td').get_text(strip=True)
        power = racquet.find('th', string='Power Level: ').find_next('td').get_text(strip=True)
        stiffness = racquet.find('th', string='Stiffness: ').find_next('td').get_text(strip=True)
        string_pattern = racquet.find('th', string='String Pattern: ').find_next('td').get_text(strip=True)
        
        # Creating a DataFrame for each set of data
        df = pd.DataFrame({
            'Racquet': [name],
            'Head Size': [head_size],
            'Balance': [balance],
            'Swing Weight': [weight],
            'Power Level': [power],
            'Stiffness': [stiffness],
            'String Pattern': [string_pattern],
        })
        
        # Adding DataFrame to the list
        dfs.append(df)

# Concatenating DataFrames into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Extracting Manufacturer from Racquet name and rearranging columns
combined_df['Manufacturer'] = combined_df['Racquet'].str.split().str[0]
combined_df.insert(0, 'Manufacturer', combined_df.pop('Manufacturer'))

# Saving the combined DataFrame to an Excel file
combined_df.to_excel('racquet_database.xlsx', index=False)

# Displaying the combined DataFrame
print(combined_df)


    Manufacturer                                  Racquet  \
0        Babolat               Babolat Boost Drive (2021)   
1        Babolat                Babolat Boost Aero Yellow   
2        Babolat                 Babolat Drive 115 (2018)   
3        Babolat                          Babolat Drive G   
4        Babolat  Babolat Pure Strike Tour 3rd Gen - 2019   
..           ...                                      ...   
270    ProKennex               ProKennex Ki Q+Tour - 2021   
271    ProKennex            ProKennex Ki 15 (260g) - 2022   
272    ProKennex              ProKennex Ki 5 (295) - 2022   
273    ProKennex            ProKennex Ki 10 (305g) - 2022   
274    ProKennex             ProKennex Ki 15 (300) (2022)   

                     Head Size  Balance Swing Weight Power Level Stiffness  \
0    105 sq. in. / 677 sq. cm.  3pts HH          312      Medium        65   
1    102 sq. in. / 658 sq. cm.  2pts HH          306  Low-Medium        67   
2    115 sq. in. / 742 sq. cm.  5